In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import glob
import os,sys
import fitsio as fio
from tqdm import tqdm
import proplot as pplt
import pickle
import glob
from astropy.io import fits
from scipy.optimize import curve_fit
import healsparse
import healpy as hp
import skyproj
import pandas as pd

%matplotlib inline

matplotlib.rcParams.update({'font.size': 20})

## Example

In [ ]:
import numpy
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.colors as colors

bands = ['Y', 'J', 'H', 'F']
bandnames = ['Y106', 'J129', 'H158', 'F184']
RR = [8.8431, 7.5760, 7.0906, 6.5169]

sigma = numpy.zeros((4,48,48))
g1 = numpy.zeros((4,48,48))
g2 = numpy.zeros((4,48,48))

X,Y = numpy.meshgrid(numpy.linspace(0,47,48), numpy.linspace(0,47,48))

# load information
fc = []
for i in range(4):
    c1 = c2 = 0
    data = numpy.loadtxt('StarCat_' + bands[i] + '.txt')
    for ix in range(48):
        for iy in range(48):
            ind = numpy.logical_and(data[:,2]==ix, data[:,3]==iy)
            c2 += numpy.count_nonzero(ind)
            ind = numpy.logical_and(ind, data[:,22]>0)
            c1 += numpy.count_nonzero(ind)
            dd = data[ind,:]
            sigma[i,iy,ix] = numpy.mean(dd[:,13])
            g1[i,iy,ix] = numpy.mean(dd[:,14])
            g2[i,iy,ix] = numpy.mean(dd[:,15])
    print(c2,c1,1-c1/c2)
    fc.append('{:4.2f}%'.format((1-c1/c2)*100))

# now make plot
matplotlib.rcParams.update({'font.size': 10})
F = plt.figure(figsize=(8,6.5))
for i in range(4):
    S = F.add_subplot(2,2,i+1)
    S.set_title(bandnames[i]) # + ': ' + fc[i] + ' cut')
    S.set_xlabel('block x')
    S.set_ylabel('block y')
    im = S.imshow(sigma[i,:,:]/(RR[i]/8*.11/.025)-1., cmap='RdBu_r', aspect=1, interpolation='nearest', origin='lower',
    norm=colors.AsinhNorm(linear_width=3e-4, vmin=-.004, vmax=.004))

    phi = 2*numpy.arctan2(g2[i,:,:],g1[i,:,:])
    g = numpy.sqrt(g1[i,:,:]**2+g2[i,:,:]**2)
    v = g/5e-4
    t = sigma[i,:,:]/(RR[i]/8*.11/.025)-1.
    t4 = numpy.mean( t.reshape((12,4,12,4)), axis=(1,3))
    print(bandnames[i], numpy.amax(v), numpy.mean(t**2)**.5, numpy.mean(t4**2)**.5)
    im2 = S.quiver(X,Y,v*numpy.cos(phi),v*numpy.sin(phi), headaxislength=0, headlength=0, headwidth=0, pivot='mid', scale=1.,
    scale_units='xy', angles='xy')
    F.colorbar(im, orientation='vertical')
F.set_tight_layout(True)
F.savefig('etmap.pdf')
plt.close(F)